In [1]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn import tree
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.metrics import precision_recall_fscore_support
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [2]:
import numpy as np
import pandas as pd

#### import the Comedians vector

In [73]:
def normalize_text(text):
    # Lowercase every words
    text = text.lower()
    # Get rid of the punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # Get rid of these specific punctuations
    text = re.sub('[‘’“”…]', '', text)
    # Get rid of '\n'
    text = re.sub('\n', '', text)
    return text

In [74]:
Comedians_vector = {'bill burr': [0.06870229, 0.59541985, 0.24427481, 0.09160305],
 'jim jefferies': [0.05806452, 0.52258065, 0.27096774, 0.1483871 ],
 'dave chappelle': [0.19369369, 0.32432432, 0.41441441, 0.06756757],
 'andy woodhull': [0.        , 0.81481481, 0.18518519, 0.        ],
 'ramy youssef': [0.        , 0.41025641, 0.41025641, 0.17948718],
 'chris rock': [0.        , 0.37820513, 0.39102564, 0.23076923],
 'jack whitehall': [0.      , 0.953125, 0.046875, 0.      ],
 'vir das': [0.        , 0.55555556, 0.22222222, 0.22222222],
 'mark normand': [0.        , 0.34042553, 0.46808511, 0.19148936],
 'bert kreischer': [0.        , 0.62857143, 0.3       , 0.07142857],
 "chris d'elia": [0.        , 0.46464646, 0.37373737, 0.16161616],
 'jimmy o. yang': [0.        , 0.56603774, 0.43396226, 0.        ],
 'hannah gadsby': [0.        , 0.7704918 , 0.13114754, 0.09836066],
 'joe rogan': [0.        , 0.42253521, 0.42253521, 0.15492958],
 'marc maron': [0.        , 0.5106383 , 0.38297872, 0.10638298],
 'whitney cummings': [0.        , 0.36904762, 0.02380952, 0.60714286],
 'aziz ansari': [0.        , 0.47457627, 0.38983051, 0.13559322],
 'neal brennan': [0.        , 0.31147541, 0.45901639, 0.2295082 ],
 'george lopez': [0.        , 0.57142857, 0.42857143, 0.        ],
 'kenny sebastian': [0.  , 0.6 , 0.15, 0.25],
 'ronny chieng': [0.    , 0.8125, 0.1875, 0.    ],
 'tom segura': [0.        , 0.41549296, 0.52112676, 0.06338028],
 'iliza shlesinger': [0.        , 0.46808511, 0.0212766 , 0.5106383 ],
 'pete davidson': [0.        , 0.46551724, 0.4137931 , 0.12068966],
 'urzila carlson': [0.        , 0.83333333, 0.13888889, 0.02777778],
 'trevor noah': [0.        , 0.44230769, 0.53846154, 0.01923077],
 'sebastian maniscalco': [0.     , 0.84375, 0.15625, 0.     ],
 'patton oswalt': [0.        , 0.75      , 0.20454545, 0.04545455],
 'kevin hart': [0.        , 0.34090909, 0.55681818, 0.10227273],
 'tiffany haddish': [0.        , 0.79166667, 0.20833333, 0.        ],
 'roy wood jr.': [0.        , 0.6744186 , 0.29069767, 0.03488372],
 'michael mcintyre': [0.        , 0.84444444, 0.13333333, 0.02222222],
 'amy schumer': [0.        , 0.50746269, 0.2238806 , 0.26865672],
 'arsenio hall': [0.        , 0.5483871 , 0.38709677, 0.06451613],
 'jo koy': [0.        , 0.53571429, 0.35714286, 0.10714286],
 'jerry seinfeld': [0.        , 0.82432432, 0.06756757, 0.10810811],
 'michelle wolf': [0.        , 0.48275862, 0.10344828, 0.4137931 ],
 'eric andre': [0.        , 0.92156863, 0.05882353, 0.01960784],
 'hasan minhaj': [0.        , 0.77419355, 0.22580645, 0.        ],
 'russell peters': [0.        , 0.52631579, 0.40350877, 0.07017544]}

In [62]:
len(Comedians_vector)

40

In [5]:
#get the comedians type
comedians_type = {}
for key,value in Comedians_vector.items():
    for i in range(4):
        if max(value) == value[i]:
            comedians_type[key] = i

#### get the useful user_comedian_matrix

In [6]:
user_comedian_matrix = pd.read_pickle('./user_comedian_matrix_OriginalPolarity.pkl')
user_comedian_matrix.columns = map(str.lower,user_comedian_matrix.columns)
user_comedian_matrix.index = map(str.lower,user_comedian_matrix.index)
user_comedian_matrix

,ramy youssef,andy woodhull,amy schumer,arsenio hall,aziz ansari,bert kreischer,bill burr,chris d'elia,chris rock,dave chappelle,...,ronny chieng,roy wood jr.,russell peters,sebastian maniscalco,tiffany haddish,tom segura,trevor noah,urzila carlson,vir das,whitney cummings
a b,0.191429,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
adedotun akande,NaN,0.875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.4,NaN,-0.1625,NaN,NaN,NaN,NaN,NaN,NaN
anton nym,NaN,0.68099,NaN,NaN,NaN,0,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lydia waweru-morgan,NaN,-0.2375,NaN,0.65,NaN,0.488281,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jennifer bates,NaN,0.671875,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
camthedestroyer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jay jay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.155,NaN,NaN,NaN,NaN,0.8,NaN,NaN,NaN,NaN
cynthia copland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.3,NaN,NaN,NaN,NaN,NaN
comedy central stand-up\n\n\n\n \n comedy central stand-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.268182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5


#### get user vector

In [7]:
#get the compplete user_comedian_matrix
user_comedian_matrix_completion = pd.read_csv('./matrix_comple_finish.csv')
user_comedian_matrix_completion = user_comedian_matrix_completion.rename(columns={"Unnamed: 0":"User"})
user_comedian_matrix_completion.set_index(["User"], inplace=True)

In [8]:
user_vector = {}
for i in range(len(user_comedian_matrix_completion)):
    temp_name = (user_comedian_matrix_completion.index)[i].lower()
    user_vector[temp_name] = [0,0,0,0]
    num_list = [0,0,0,0]
    for j in range(user_comedian_matrix_completion.shape[1]):
        temp_com_name = list(user_comedian_matrix_completion)[j].lower()
        if temp_com_name in Comedians_vector:
            temp_index = comedians_type[temp_com_name]
            user_vector[temp_name][temp_index] += user_comedian_matrix_completion.iloc[i,j]
            num_list[temp_index] += 1
    for k in range(4):
        if  num_list[k] != 0:
            user_vector[temp_name][k] /= num_list[k]
user_vector

{'a b': [0,
  2.5447527209618303e-06,
  0.021273133940624984,
  1.764290388076519e-05],
 'adedotun akande': [0,
  0.07247430468513961,
  0.05422352403192965,
  0.290510601277332],
 'anton nym': [0,
  0.06369994738524244,
  0.053393395559451134,
  0.3119818083578212],
 'lydia waweru-morgan': [0,
  -0.011376068476211436,
  -0.02378584498041533,
  -0.12743620505153583],
 'jennifer bates': [0,
  0.06354776005623838,
  0.06515098529830295,
  0.3490561015856397],
 'r. a.': [0,
  0.005601835088384821,
  -0.020108954021497386,
  -0.10773671442681287],
 'ethan hayden': [0,
  0.02537315966484946,
  0.013040362454482469,
  0.11849378265895749],
 'dan lowe': [0,
  0.004923843355997424,
  0.004000220378471845,
  0.021431776118191494],
 'sumukh': [0,
  0.009811140327325791,
  0.049358529930785135,
  0.04411904461095983],
 'a': [0, 0.10433073006421299, 0.07474865019417777, 0.50042494157543],
 'brainflash1': [0,
  -0.0007243489392432858,
  0.0070550961375529025,
  0.11759154122988638],
 'babychaingang

In [63]:
len(user_vector)

79

#### get the train set and test set

In [59]:
train_set = pd.read_csv('./training_set.csv')
train_set = train_set[{"Name","Comedian"}]
test_set = pd.read_csv('./test_set.csv')
test_set = test_set[{"Name","Comedian"}]

In [75]:
dfnow = train_set.copy()
dfnow['Comedian'] = dfnow['Comedian'].apply(lambda x:x.lower())

In [76]:
dfnow[~dfnow.Comedian.isin(Comedians_vector.keys())]

,Comedian,Name


In [ ]:
train_set[~train_set.User.isin(Comedians_vector.keys())]

In [77]:
train_data = pd.DataFrame(columns = ["prefer1","prefer2","prefer3","prefer4","style1","style2","style3","style4","polarity"])
test_data = pd.DataFrame(columns = ["prefer1","prefer2","prefer3","prefer4","style1","style2","style3","style4","polarity"])

In [78]:
for i in range(len(train_set)):
    temp_com_name = train_set.iloc[i,0].lower()
    temp_user_name = train_set.iloc[i,1].lower()
    if temp_com_name in Comedians_vector and temp_user_name in user_vector:
        temp_com_list = Comedians_vector[temp_com_name]
        temp_user_list = user_vector[temp_user_name]
        train_data = train_data.append({"prefer1":temp_user_list[0], "prefer2":temp_user_list[1],
                                        "prefer3": temp_user_list[2], "prefer4": temp_user_list[3], 
                                        "style1":temp_com_list[0],"style2":temp_com_list[1],
                                        "style3":temp_com_list[2],"style4":temp_com_list[3],
                                        "polarity":user_comedian_matrix.loc[temp_user_name,temp_com_name]},
                                       ignore_index=True)
train_data

,prefer1,prefer2,prefer3,prefer4,style1,style2,style3,style4,polarity
0,0.0,0.000003,0.021273,0.000018,0.0,0.410256,0.410256,0.179487,0.191429
1,0.0,0.000003,0.021273,0.000018,0.0,0.555556,0.222222,0.222222,0.000000
2,0.0,0.072474,0.054224,0.290511,0.0,0.843750,0.156250,0.000000,-0.162500
3,0.0,0.072474,0.054224,0.290511,0.0,0.814815,0.185185,0.000000,0.875000
4,0.0,0.063700,0.053393,0.311982,0.0,0.628571,0.300000,0.071429,0.000000
...,...,...,...,...,...,...,...,...,...
175,0.0,-0.003453,-0.003838,-0.020562,0.0,0.465517,0.413793,0.120690,0.291667
176,0.0,0.038314,0.037272,0.422790,0.0,0.369048,0.023810,0.607143,0.500000
177,0.0,0.038314,0.037272,0.422790,0.0,0.465517,0.413793,0.120690,0.268182
178,0.0,0.033821,0.030205,0.161826,0.0,0.791667,0.208333,0.000000,0.000000


In [79]:
for i in range(len(test_set)):
    temp_com_name = test_set.iloc[i,0].lower()
    temp_user_name = test_set.iloc[i,1].lower()
    if temp_com_name in Comedians_vector and temp_user_name in user_vector:
        temp_com_list = Comedians_vector[temp_com_name.lower()]
        temp_user_list = user_vector[temp_user_name.lower()]
        test_data = test_data.append({"prefer1":temp_user_list[0], "prefer2":temp_user_list[1],
                                        "prefer3": temp_user_list[2], "prefer4": temp_user_list[3], 
                                        "style1":temp_com_list[0],"style2":temp_com_list[1],
                                        "style3":temp_com_list[2],"style4":temp_com_list[3],
                                        "polarity":user_comedian_matrix.loc[temp_user_name,temp_com_name]},
                                       ignore_index=True)

In [80]:
test_data.loc[test_data['polarity'] >= 0,'polarity' ] = 1
test_data.loc[test_data['polarity'] < 0,'polarity' ] = -1
train_data.loc[train_data['polarity'] >= 0,'polarity' ] = 1
train_data.loc[train_data['polarity'] < 0,'polarity' ] = -1

In [82]:
X_train = train_data.iloc[:,0:8]
X_test = test_data.iloc[:,0:8]
y_train = train_data[["polarity"]]
y_test = test_data[["polarity"]]

In [83]:
clf = KNeighborsClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
pd.DataFrame(confusion_matrix(y_test, y_pred))

C:\python\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


,0,1
0,0,11
1,2,67


In [84]:
pre = metrics.precision_score(y_test, y_pred)
rec = metrics.recall_score(y_test, y_pred)
score = clf.score(X_test,y_test)
print("precision:", pre, "\nrecall:", rec, "\nscore:", score)

precision: 0.8589743589743589 
recall: 0.9710144927536232 
score: 0.8375


In [85]:
clf = linear_model.SGDClassifier(loss='squared_loss')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
pd.DataFrame(confusion_matrix(y_test, y_pred))

C:\python\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


,0,1
0,8,3
1,36,33


In [86]:
pre = metrics.precision_score(y_test, y_pred)
rec = metrics.recall_score(y_test, y_pred)
score = clf.score(X_test,y_test)
print("precision:", pre, "\nrecall:", rec, "\nscore:", score)

precision: 0.9166666666666666 
recall: 0.4782608695652174 
score: 0.5125


In [87]:
clf = RandomForestClassifier(n_estimators=8)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
pd.DataFrame(confusion_matrix(y_test, y_pred))

C:\python\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


,0,1
0,0,11
1,7,62


In [88]:
pre = metrics.precision_score(y_test, y_pred)
rec = metrics.recall_score(y_test, y_pred)
score = clf.score(X_test,y_test)
print("precision:", pre, "\nrecall:", rec, "\nscore:", score)

precision: 0.8493150684931506 
recall: 0.8985507246376812 
score: 0.775


In [89]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
pd.DataFrame(confusion_matrix(y_test, y_pred))

,0,1
0,1,10
1,11,58


In [90]:
pre = metrics.precision_score(y_test, y_pred)
rec = metrics.recall_score(y_test, y_pred)
score = clf.score(X_test,y_test)
print("precision:", pre, "\nrecall:", rec, "\nscore:", score)

precision: 0.8529411764705882 
recall: 0.8405797101449275 
score: 0.7375


In [91]:
clf = SVC(kernel='rbf', probability=True)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
pd.DataFrame(confusion_matrix(y_test, y_pred))

C:\python\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


,0,1
0,0,11
1,0,69


In [92]:
pre = metrics.precision_score(y_test, y_pred)
rec = metrics.recall_score(y_test, y_pred)
score = clf.score(X_test,y_test)
print("precision:", pre, "\nrecall:", rec, "\nscore:", score)

precision: 0.8625 
recall: 1.0 
score: 0.8625
